In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dense
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas_datareader import data as pdr
import yfinance as yf

yf.pdr_override()

In [2]:
now = datetime.now()

before = now - relativedelta(years = 10)

now_day = now.strftime("%Y-%m-%d")
before_day = before.strftime("%Y-%m-%d")
print(f"end : {now_day}")
print(f"start : {before_day}")

samsung_stock = pdr.get_data_yahoo("005930.KS", start=before_day, end=now_day)
print(samsung_stock)

end : 2022-11-27
start : 2012-11-27
[*********************100%***********************]  1 of 1 completed
               Open     High      Low    Close     Adj Close    Volume
Date                                                                  
2012-11-27  28100.0  28560.0  28080.0  28320.0  22659.490234  11836200
2012-11-28  28000.0  28760.0  27920.0  28260.0  22611.482422  14055800
2012-11-29  28040.0  28440.0  28040.0  28280.0  22627.486328  13073300
2012-11-30  28460.0  28500.0  27980.0  28120.0  22499.470703  16427600
2012-12-03  28400.0  28720.0  28160.0  28600.0  22883.525391  10220850
...             ...      ...      ...      ...           ...       ...
2022-11-21  61400.0  61800.0  60800.0  61400.0  61400.000000   9378097
2022-11-22  60900.0  61200.0  60300.0  60600.0  60600.000000   9411289
2022-11-23  61200.0  61300.0  60700.0  61000.0  61000.000000   8765050
2022-11-24  61200.0  61700.0  60900.0  61400.0  61400.000000   8125010
2022-11-25  61400.0  61700.0  60800.0  6100

In [3]:
close_pirces = samsung_stock['Close'].values
print(close_pirces)

window_size = 30

result_list = []
for i in range(len(close_pirces) - (window_size + 1)):
    result_list.append(close_pirces[i: i + (window_size + 1)])

normal_data = []
for window in result_list:
    window_list = [((float(p)/float(window[0]))-1)for p in window]
    normal_data.append(window_list)

result_list = np.array(normal_data)
print(result_list.shape[0], result_list.shape[1])

[28320. 28260. 28280. ... 61000. 61400. 61000.]
2425 31


In [4]:
row = int(round(result_list.shape[0]*0.9))
train = result_list[:row, :]

x_train = train[:, :-1]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1]))
y_train = train[:, -1]

x_test = result_list[row:, :-1]
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1]))
y_test = result_list[row:, -1]

x_train.shape, x_test.shape

((2182, 30), (243, 30))

In [ ]:
model = Sequential()
model.add(LSTM(window_size, return_sequences=True, input_shape=(window_size, 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse',optimizer='rmsprop')
model.summary()